Logistic Regression: HS vs Controls

In [ ]:
# --- Load required libraries ---
library(dplyr)
library(tidyverse)
library(bigrquery)
library(MatchIt)
library(lubridate)
library(data.table)

In [ ]:
matrix <- read.csv("df_final_matrix.csv") 

In [ ]:
# --- Define Phecode features
phecodes <- setdiff(names(matrix), c('person_id','HS','Asian',
                                     'Black.or.African.American',
                                     'More.than.one.population',
                                     'No.answer.race',
                                     'Other.race',
                                     'White',
                                     'Female',
                                     'Male',
                                     'Other.sex',
                                     'Hispanic.or.Latino',
                                     'No.answer.ethinicity',
                                     'Not.Hispanic.or.Latino',
                                     'Other.ethinicity',
                                     'age_last_EHR',
                                     'age_normalized'))

In [ ]:
# --- Compute sample weights to address class imbalance ---
fraction_0 <- rep(1 - (sum(matrix$HS == 0))/nrow(matrix))
fraction_1 <- rep(1 - (sum(matrix$HS == 1))/nrow(matrix))
matrix$weight <- ifelse(matrix$HS == 0, fraction_0, fraction_1)

In [ ]:
# --- Run weighted logistic regression for each phecode ---
start_time <- Sys.time()
#
logit <- lapply(phecodes, function(phecode) {
  formula_str <- paste("HS ~ `", phecode, "`+ Asian + Black.or.African.American + More.than.one.population + No.answer.race + Other.race + White + Female + Male + Other.sex + Hispanic.or.Latino + No.answer.ethinicity + Not.Hispanic.or.Latino + Other.ethinicity + age_normalized", sep="")
  model <- glm(formula=as.formula(formula_str), data = matrix, family = binomial, weights = matrix$weight) 
  s <- summary(model)
  ci <- confint.default(model)[phecode, ]
  list(s,ci)
})

end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
# --- Extract model outputs ---
beta_vals  <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[1]]$coefficients[2, 1]))
std_vals   <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[1]]$coefficients[2, 2]))
z_vals     <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[1]]$coefficients[2, 3]))
p_values   <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[1]]$coefficients[2, 4]))

lower_ci   <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[2]][1]))
upper_ci   <- unlist(lapply(logit, function(res) if (is.null(res)) NA else res[[2]][2]))
adjusted_pvalues_bh <- p.adjust(p_values, method = "BH")

In [ ]:
plot_data <- data.frame(
    Phecode  = phecodes, 
    Beta     = beta_vals,
    Std      = std_vals,
    Z_score  = z_vals,
    lower_CI = lower_ci,
    upper_CI = upper_ci,    
    Pvalue   = p_values,
    Adjust_PV = adjusted_pvalues_bh
)
plot_data$NegativeLogP = -log10(plot_data$Adjust_PV)
plot_data$BetaDirection <- ifelse(plot_data$Beta > 0, "Positive", "Negative")
head(plot_data)

In [ ]:
# --- Output results ---
write.csv(plot_data, "plot_data_df.csv", row.names = FALSE)